# Run the final state algorithm on a ROOT HSDATA file

Here we create the topology particle trees using TreePrepManager. These trees contain the particle variables to be used in the MVA. You can configure which variables you would like to use for each particle when declaring the TreePrep. Once the final state has perocessed the events the particle trees can be used to train a TMVA classifier via hsmva interface.

**Important** The training of signal and background requires these types of events to be flagged. For simulated events this is done automatically by setting the fCorrect variable to true(sig) or false(bg). For this to work the user needs to link the reconstructed particles to their truth values. this is done at the AddParticle stage of the FinalState constructor. For this example you will find the lines :

  AddParticle("Beam",&fBeam,kTRUE,-1);
  AddParticle("Proton",&fProton,kTRUE,0);
  AddParticle("Kp",&fKp,kTRUE,1);
  AddParticle("Km",&fKm,kTRUE,2);

-1=> beam does not have a truth particle; 0=>proton first truth particle in Generated branch; 1=>Kp second....

The Correct truth algorithm is not 100% garunteed. You may wish to make it stricter by applyiny a cut on the angle difference (true - reconstructed), by adding the line SetAccurateTruth(X); where X is the angle diff cut in degrees. (Currently this must be done in K2.C or equivalent)

In [ ]:
gROOT->ProcessLine(".x LoadK2.C+");

First create an object of your final state class. You may configure it if you want to limit the number of particles in an event or set to just run on generated simulation data.
This notebook is equivalent to the RunFSRootXXX.C macro which will be in the code directory.
Again just change XXX for your class name.

In [ ]:
 K2 fs("Beam","Beam");
  // fs.SetGenerated(); //just analyse generated branch
  // fs.SetMaxParticles(10); //max number of particles of any 1 type
 

Create an output tree. Here we will use a FiledTree object which is a wrapper class to a tree in a file. We will also create this as a smart pointer so we can delete it at the end by calling reset(), this will save the tree to disk. The Filed tree needs 2 strings ,1 a treename and the other the file name. You should give the full path for where you would like the files saved. It will overwrite any existing files. 

In [ ]:
auto output = new HS::FiledTree("tt","testcuts.root");
fs.FinalStateOutTree(output->Tree()); //connect ouput tree to final state branches

Here we add the cuts for each particle. This is done via a HS::Cuts object which gets propogated to each particle in each topology. To specify any type of cut behaviour you need to generate your own Cuts Method. Here we just use a predefined delta time cut but you could make your won class which has momentum dependence as well.

**Making your own cuts class** Here I use a new cuts class DeltaTimePCut. You can make your own cuts follwing the example notebook [DeltaTimePCut](MakeDeltaTimePCut.ipynb)

In [ ]:
ParticleCutsManager cutsman;
auto beamCut=make_shared<DeltaTimePCut>(20,3); //7ns delta time cut
auto otherCut=make_shared<DeltaTimePCut>(20,3); //10ns cut for all other particles
cutsman.AddParticleCut("Beam",beamCut); //assign to manager
cutsman.SetDefaultCut(otherCut); //assign to manager
cutsman.ConfigureCuts(&fs);    //propogate the cuts to topologies in fs

Now register the cuts manager as a post topology function action. This will be applied after the Topo_0 etc functions when start times etc have been determined. 

In [ ]:
fs.RegisterPostTopoAction(&cutsman);

**Particle Monitoring** here we are going to make seperate trees for each topology and only keep the detected particle information in each one.

In [ ]:
auto treeman=new TreePrepManager("/scratch/dglazier/g8/particles/t3MVAMLP");
treeman->SetParticleVars("Beam",{"P","Time"}); //For beam only interesetef in energy and time
treeman->ConfigureTreeParticles(&fs); //propogate through topologies
//treeman.SetN(50000); //Number of events required for each topo. Number of train+test events must be less than this
fs.RegisterPostWorkAction(treeman); //register post-work i.e. after kinematics is called and all calculations

Now we configure the input data. Here we use a ROOT file presaved in HSDATA format. It is possible to analyse other dataformats via the DataManager class.

In [ ]:
 //create datamanager
  auto dm=std::make_shared<DataManager>();

  //And make a chain of data files
  TChain chain("HSParticles");
  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048088.dat.root");
  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048089.dat.root");
//  chain.Add("/w/work1/jlab/hallb/HS_g8/1_9_list/flist_048090.dat.root");

  chain.Add("/w/work1/jlab/hallb/HSg8/g8sim_2k_b/rec_edgen_.root");
   dm->InitChain(&chain);

  //connect FinalState to Data by moving the pointer
  fs.SetDataManager(dm);

Analyse all the events. Not you may prefer just looping over the data manager directly via while(dm->ReadEvent()) fs->Process();
You can also supply a number of events to analyse e.g. fs.ProcessData(100);

In [ ]:
cout<<"Number of Events to process "<<chain.GetEntries()<<endl;

fs.ProcessData(); //No number give, analyse all events in chain
cout<<"Done "<<dm->GetEntry()<< " events "<<endl;

And remember to save the tree!

In [ ]:
//delete output;

In [ ]:
%jsroot on

In [ ]:
TCanvas c3;
c3.Divide(2,2);
c3.cd(1);
treeman->GetPrep(0)->Tree()->Draw("BeamP:BeamTime>>h1(100,-10,10,100,0.4)","","col1");
c3.cd(2);
treeman->GetPrep(0)->Tree()->Draw("ProtonP:ProtonTime>>h2(100,-10,10,100,0,4)","","col1");
c3.cd(3);
treeman->GetPrep(0)->Tree()->Draw("KpP:KpTime>>h3(100,-10,10,100,0,4)","","col1");
c3.cd(4);
treeman->GetPrep(0)->Tree()->Draw("KmP:KmTime>>h4(100,-10,10,100,0,4)","","col1");


In [ ]:
c3.Draw();

Now train the MVA classifiers

In [ ]:
gROOT->ProcessLine(".x $HSCODE/hsmva/LoadHSMVA.C");

In [ ]:
 for(UInt_t ip=0;ip<treeman->NPreps();ip++ ){ //Loop over topologies
   auto prep=treeman->GetPrep(ip);
   cout<<prep<<endl;
   auto trainer=  new TrainSignalID(Form("Topo%d",ip));
     
   trainer->SetOutDir(treeman->OutDir());
   trainer->IgnoreBranches("Signal:Background"); //ignore these branches for training
   trainer->AddSignalTree(prep->Tree(),"Signal"); //Use "Signal" branch to filter Signal events
   trainer->AddBackgroundTree(prep->Tree(),"Background"); //Use "Background" branch for bg events
   trainer->SetNTrainTest(50000,50000); //set number of training and testing events, there must be sufficient number in the trees for each topo
   trainer->PrepareTrees(); 
   //trainer->BookMethod(Meths.BDTA); //choose some classifiers
   trainer->BookMethod(Meths.MLP); //choose some classifiers
   trainer->DoTraining();
   trainer->DrawROCCurve();
   //trainer->DrawResponses();
   delete trainer;
  }

**When ready delete the treeman so trees as saved to the output directory**

In [ ]:
delete treeman;